In [1]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train_original=pd.read_csv('train.csv')
train_original.set_index('PassengerId',inplace=True)

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import powerlaw
from scipy.stats import uniform


def create_model(in_layer,neurons=128,activation='relu',dropout=0.25,optimizer='adam',shape='one'):
    model=None
    model = Sequential()
    #first, input, layer, always present
    model.add(Dense(neurons, input_dim=in_layer,activation=activation))
    model.add(Dropout(dropout))
    if(shape=='none'):
        None
    if(shape=='one'):
        model.add(Dense(neurons,activation=activation))
        model.add(Dropout(dropout))
    if(shape=='two'):
        model.add(Dense(neurons,activation=activation))
        model.add(Dropout(dropout))
        model.add(Dense(neurons,activation=activation))
        model.add(Dropout(dropout))
    #final layer, always present
    model.add(Dense(8, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [180]:
data_1 = pd.read_csv('data_raw_1.csv',index_col='PassengerId')
data_2 = pd.read_csv('data_raw_2.csv',index_col='PassengerId')
data_3 = pd.read_csv('data_raw_3.csv',index_col='PassengerId')

decks=['A_deck', 'B_deck', 'C_deck',
       'D_deck', 'E_deck', 'F_deck', 'G_deck', 'T_deck']
decks_survived=['A_deck', 'B_deck', 'C_deck',
       'D_deck', 'E_deck', 'F_deck', 'G_deck', 'T_deck','Survived','Missing_Embark','Missing_Deck','deck']



deck_keys=pd.DataFrame({'A_deck':[1,0,0,0,0,0,0,0],
                        'B_deck':[0,1,0,0,0,0,0,0],
                        'C_deck':[0,0,1,0,0,0,0,0],
                        'D_deck':[0,0,0,1,0,0,0,0],
                        'E_deck':[0,0,0,0,1,0,0,0],
                        'F_deck':[0,0,0,0,0,1,0,0],
                        'G_deck':[0,0,0,0,0,0,1,0],
                        'T_deck':[0,0,0,0,0,0,0,1],
                        'Missing_Deck':[0,0,0,0,0,0,0,0]})

deck_keys_from_int = {
    '0':'A_deck',
    '1':'B_deck',
    '2':'C_deck',
    '3':'D_deck',
    '4':'E_deck',
    '5':'F_deck',
    '6':'G_deck',
    '7':'T_deck'
}

def get_deck(row):
    for c in deck_keys.columns:
        if row[c]==1:
            return c

data_1['deck']=data_1.apply(get_deck,axis=1)
data_1_nodeck=data_1.loc[(data_1['deck'] == 'Missing_Deck')]
data_1_deck=data_1.loc[(data_1['deck'] != 'Missing_Deck')]

data_2['deck']=data_2.apply(get_deck,axis=1)
data_2_nodeck=data_2.loc[(data_2['deck'] == 'Missing_Deck')]
data_2_deck=data_2.loc[(data_2['deck'] != 'Missing_Deck')]

data_3['deck']=data_3.apply(get_deck,axis=1)
data_3_nodeck=data_3.loc[(data_3['deck'] == 'Missing_Deck')]
data_3_deck=data_3.loc[(data_3['deck'] != 'Missing_Deck')]

print(data_1.shape,"\n",data_2.shape,"\n",data_3.shape)

(1309, 21) 
 (1309, 27) 
 (1309, 31)


# Data 1

In [6]:
X=data_1_deck.drop(decks_survived,axis=1)
Y=data_1_deck[decks].as_matrix()#.reshape((-1,1))

classifier = None
classifier = KerasClassifier(build_fn=create_model,verbose=0)

param_distributions = {'epochs':[1000],
                       'batch_size':[1024],
                       'in_layer':[X.shape[1]],
                       'neurons' : [256],#[32,64,128],
                       'optimizer': ['adam'],#['adam','adagrad','rmsprop','adadelta'],
                       'dropout' : np.linspace(0.3,0.7,11),#uniform(0.1,1)
                       'activation' : ['relu'],
                       'shape':['two']
                      }

rscv = RandomizedSearchCV(estimator=classifier,
                          param_distributions=param_distributions,
                          cv=5,
                          verbose=1,
                          n_iter=10)

rscv.fit(X,Y)
results_df=pd.DataFrame(rscv.cv_results_)

results_df.sort_values('rank_test_score',axis=0).drop(['param_epochs','std_score_time','std_fit_time','params','mean_score_time','split0_test_score',
       'split0_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score','param_batch_size','param_in_layer'],axis=1).head(10)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 10.0min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_dropout,param_neurons,param_optimizer,param_shape,rank_test_score,std_test_score,std_train_score
9,11.543897,0.403390,0.866949,relu,0.42,256,adam,two,1,0.049589,0.013292
4,10.124330,0.396610,0.773729,relu,0.58,256,adam,two,2,0.054237,0.022357
8,11.251261,0.396610,0.801695,relu,0.54,256,adam,two,2,0.057327,0.020999
3,10.160644,0.396610,0.686441,relu,0.66,256,adam,two,4,0.057327,0.018372
7,10.943918,0.393220,0.890678,relu,0.38,256,adam,two,5,0.024910,0.011804
1,9.349178,0.386441,0.900847,relu,0.3,256,adam,two,6,0.042067,0.007388
2,9.562164,0.386441,0.818644,relu,0.5,256,adam,two,6,0.034570,0.018644
6,10.972853,0.386441,0.889831,relu,0.34,256,adam,two,6,0.029160,0.005992
5,10.453851,0.383051,0.645763,relu,0.7,256,adam,two,9,0.061205,0.021706
0,9.437125,0.359322,0.848305,relu,0.46,256,adam,two,10,0.045982,0.012397


## Predicting deck

In [185]:
X=data_1_deck.drop(decks_survived,axis=1)
Y=data_1_deck[decks].as_matrix()#.reshape((-1,1))

classifier = None
classifier = create_model(in_layer=X.shape[1],neurons=256,dropout=0.5,shape='two')
classifier.fit(X,Y,epochs=5000,verbose=0,batch_size=1024)


In [186]:
predicteddeck_df=pd.DataFrame(classifier.predict_classes(data_1_nodeck.drop(decks_survived,axis=1),batch_size=1024),index=data_2_nodeck.index,columns=['PredictedDeck'])


predicteddeck_df['deck']=predicteddeck_df.astype(str).replace(deck_keys_from_int)

data_1_nodeck_rep=data_1_nodeck.drop(decks,axis=1).drop('Missing_Deck',axis=1).copy()
data_1_deck_rep=data_1_deck.drop(decks,axis=1).drop('Missing_Deck',axis=1).copy()
data_1_nodeck_rep['deck']=predicteddeck_df['deck']

data_1_decks_pred=pd.concat([data_1_deck_rep,data_1_nodeck_rep]).sort_index()

deck_dummies=pd.get_dummies(data_1_decks_pred['deck'],drop_first=False)

data_1_decks_pred=pd.concat([data_1_decks_pred,deck_dummies],axis=1).drop('deck',axis=1)

train_1_deck_pred=data_1_decks_pred[data_1_decks_pred['Survived']<2]
test_1_deck_pred=data_1_decks_pred[data_1_decks_pred['Survived']==2].drop('Survived',axis=1)

train_1_deck_pred.to_csv("./data_train_deck_pred_1.csv")#Train
test_1_deck_pred.to_csv("./data_test_deck_pred_1.csv")#Test

# Data 2

In [7]:
X=data_2_deck.drop(decks_survived,axis=1)
Y=data_2_deck[decks].as_matrix()#.reshape((-1,1))

classifier = None
classifier = KerasClassifier(build_fn=create_model,verbose=0)

param_distributions = {'epochs':[1000],
                       'batch_size':[1024],
                       'in_layer':[X.shape[1]],
                       'neurons' : [256],#[32,64,128],
                       'optimizer': ['rmsprop'],#['adam','adagrad','rmsprop','adadelta'],
                       'dropout' : np.linspace(0.3,0.7,11),
                       'activation' : ['relu'],
                       'shape':['two']
                      }

rscv = RandomizedSearchCV(estimator=classifier,
                          param_distributions=param_distributions,
                          cv=5,
                          verbose=1,
                          n_iter=10)

rscv.fit(X,Y)
results_df=pd.DataFrame(rscv.cv_results_)

results_df.sort_values('rank_test_score',axis=0).drop(['param_epochs','std_score_time','std_fit_time','params','mean_score_time','split0_test_score',
       'split0_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score','param_batch_size','param_in_layer'],axis=1).head(10)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 12.6min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_dropout,param_neurons,param_optimizer,param_shape,rank_test_score,std_test_score,std_train_score
1,11.599699,0.440678,0.851695,relu,0.62,256,rmsprop,two,1,0.033898,0.011681
7,13.547408,0.437288,0.883898,relu,0.58,256,rmsprop,two,2,0.042067,0.021705
3,12.138423,0.433898,0.818644,relu,0.66,256,rmsprop,two,3,0.043675,0.017034
5,12.514424,0.427119,0.934746,relu,0.46,256,rmsprop,two,4,0.034570,0.009883
6,13.184444,0.423729,0.780508,relu,0.7,256,rmsprop,two,5,0.037134,0.015488
2,11.897243,0.413559,0.916949,relu,0.5,256,rmsprop,two,6,0.022991,0.012455
0,12.000760,0.403390,0.939831,relu,0.42,256,rmsprop,two,7,0.047215,0.003171
9,13.261916,0.403390,0.960169,relu,0.34,256,rmsprop,two,8,0.061017,0.006885
4,12.302268,0.400000,0.951695,relu,0.38,256,rmsprop,two,9,0.036510,0.010585
8,13.031654,0.400000,0.905932,relu,0.54,256,rmsprop,two,10,0.036510,0.020304


## Predicting deck

In [187]:
X=data_2_deck.drop(decks_survived,axis=1)
Y=data_2_deck[decks].as_matrix()#.reshape((-1,1))

classifier = None
classifier = create_model(in_layer=X.shape[1],neurons=256,dropout=0.5,shape='two',optimizer='rmsprop')
classifier.fit(X,Y,epochs=5000,verbose=0,batch_size=1024)

In [176]:
predicteddeck_df=pd.DataFrame(classifier.predict_classes(data_2_nodeck.drop(decks_survived,axis=1),batch_size=1024),index=data_2_nodeck.index,columns=['PredictedDeck'])

predicteddeck_df['deck']=predicteddeck_df.astype(str).replace(deck_keys_from_int)

data_2_nodeck_rep=data_2_nodeck.drop(decks,axis=1).drop('Missing_Deck',axis=1).copy()
data_2_deck_rep=data_2_deck.drop(decks,axis=1).drop('Missing_Deck',axis=1).copy()
data_2_nodeck_rep['deck']=predicteddeck_df['deck']

data_2_decks_pred=pd.concat([data_2_deck_rep,data_2_nodeck_rep]).sort_index()

deck_dummies=pd.get_dummies(data_2_decks_pred['deck'],drop_first=False)

data_2_decks_pred=pd.concat([data_2_decks_pred,deck_dummies],axis=1).drop('deck',axis=1)

train_2_deck_pred=data_2_decks_pred[data_2_decks_pred['Survived']<2]
test_2_deck_pred=data_2_decks_pred[data_2_decks_pred['Survived']==2].drop('Survived',axis=1)

train_2_deck_pred.to_csv("./data_train_deck_pred_2.csv")#Train
test_2_deck_pred.to_csv("./data_test_deck_pred_2.csv")#Test

# Data 3

In [8]:
X=data_3_deck.drop(decks_survived,axis=1)
Y=data_3_deck[decks].as_matrix()#.reshape((-1,1))

classifier = None
classifier = KerasClassifier(build_fn=create_model,verbose=0)


param_distributions = {'epochs':[1000],
                       'batch_size':[1024],
                       'in_layer':[X.shape[1]],
                       'neurons' : [256],#[32,64,128],
                       'optimizer': ['adadelta'],#['adam','adagrad','rmsprop','adadelta'],
                       'dropout' : np.linspace(0.3,0.7,11),
                       'activation' : ['relu'],
                       'shape':['two']
                      }

rscv = RandomizedSearchCV(estimator=classifier,
                          param_distributions=param_distributions,
                          cv=5,
                          verbose=1,
                          n_iter=10)

rscv.fit(X,Y)
results_df=pd.DataFrame(rscv.cv_results_)

results_df.sort_values('rank_test_score',axis=0).drop(['param_epochs','std_score_time','std_fit_time','params','mean_score_time','split0_test_score',
       'split0_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score','param_batch_size','param_in_layer'],axis=1).head(10)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 16.2min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_dropout,param_neurons,param_optimizer,param_shape,rank_test_score,std_test_score,std_train_score
0,15.265613,0.410169,0.881356,relu,0.58,256,adadelta,two,1,0.032866,0.012570
8,16.705515,0.403390,0.857627,relu,0.62,256,adadelta,two,2,0.045982,0.012455
9,17.497077,0.400000,0.950847,relu,0.34,256,adadelta,two,3,0.038051,0.009513
6,16.111731,0.396610,0.789831,relu,0.7,256,adadelta,two,4,0.065731,0.010240
7,16.764510,0.396610,0.922881,relu,0.5,256,adadelta,two,4,0.057327,0.004941
5,15.872494,0.393220,0.818644,relu,0.66,256,adadelta,two,6,0.073019,0.012397
4,15.565036,0.389831,0.939830,relu,0.42,256,adadelta,two,7,0.028361,0.013238
1,14.740075,0.383051,0.957627,relu,0.3,256,adadelta,two,8,0.022991,0.008888
3,15.256824,0.379661,0.927966,relu,0.46,256,adadelta,two,9,0.044971,0.011050
2,15.436858,0.366102,0.953390,relu,0.38,256,adadelta,two,10,0.038051,0.011049


## Predicting deck

In [188]:
X=data_3_deck.drop(decks_survived,axis=1)
Y=data_3_deck[decks].as_matrix()#.reshape((-1,1))

classifier = None
classifier = create_model(in_layer=X.shape[1],neurons=256,dropout=0.5,shape='two',optimizer='rmsprop')
classifier.fit(X,Y,epochs=5000,verbose=0,batch_size=1024)

In [176]:
predicteddeck_df=pd.DataFrame(classifier.predict_classes(data_3_nodeck.drop(decks_survived,axis=1),batch_size=1024),index=data_2_nodeck.index,columns=['PredictedDeck'])

predicteddeck_df['deck']=predicteddeck_df.astype(str).replace(deck_keys_from_int)

data_3_nodeck_rep=data_3_nodeck.drop(decks,axis=1).drop('Missing_Deck',axis=1).copy()
data_3_deck_rep=data_3_deck.drop(decks,axis=1).drop('Missing_Deck',axis=1).copy()
data_3_nodeck_rep['deck']=predicteddeck_df['deck']

data_3_decks_pred=pd.concat([data_3_deck_rep,data_3_nodeck_rep]).sort_index()

deck_dummies=pd.get_dummies(data_3_decks_pred['deck'],drop_first=False)

data_3_decks_pred=pd.concat([data_3_decks_pred,deck_dummies],axis=1).drop('deck',axis=1)

train_3_deck_pred=data_3_decks_pred[data_3_decks_pred['Survived']<2]
test_3_deck_pred=data_3_decks_pred[data_3_decks_pred['Survived']==2].drop('Survived',axis=1)

train_3_deck_pred.to_csv("./data_train_deck_pred_3.csv")#Train
test_3_deck_pred.to_csv("./data_test_deck_pred_3.csv")#Test